<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Sources" data-toc-modified-id="Data-Sources-1">Data Sources</a></span></li><li><span><a href="#pyspark.sql-vs-SQL" data-toc-modified-id="pyspark.sql-vs-SQL-2"><code>pyspark.sql</code> vs SQL</a></span><ul class="toc-item"><li><span><a href="#Order-of-execution" data-toc-modified-id="Order-of-execution-2.1">Order of execution</a></span></li><li><span><a href="#Using-SQL-queries-on-a-data-frame" data-toc-modified-id="Using-SQL-queries-on-a-data-frame-2.2">Using SQL queries on a data frame</a></span></li><li><span><a href="#Table-vs-View-concept" data-toc-modified-id="Table-vs-View-concept-2.3">Table vs View concept</a></span></li></ul></li><li><span><a href="#Using-the-Spark-catalog-for-multiple-views" data-toc-modified-id="Using-the-Spark-catalog-for-multiple-views-3">Using the Spark catalog for multiple views</a></span><ul class="toc-item"><li><span><a href="#Data-Source---Backblaze-Data-Set" data-toc-modified-id="Data-Source---Backblaze-Data-Set-3.1">Data Source - Backblaze Data Set</a></span></li></ul></li></ul></div>

# Bilingual PySpark: blending Python and SQL

> This chapter is dedicated to using SQL with, and on top of PySpark. I cover how we can move from one language to the other. I also cover how we can use a SQL-like syntax within data frame methods to speed up your code and some of trade-offs you can face. Finally, we blend Python and SQL code together to get the best of both worlds.

## Data Sources

We will be using a periodic table of elements database for the initial section, followed by a public data set provided by BackBlaze, which provides hard drive data and statistics.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pyspark.sql.types as T
import numpy as np

spark = SparkSession.builder.getOrCreate()

In [2]:
# Read in table of elements data
elements = spark.read.csv(
    "data/Ch07/Periodic_Table_Of_Elements.csv",
    header=True,
    inferSchema=True,
)

# Inspect the data frame
elements.printSchema()

# View the data frame in chunks of 3-4 columns
# column_split = np.array_split(np.array(elements.columns), len(elements.columns) // 3)

# for x in column_split:
#     elements.select(*x).show(3, False)

root
 |-- AtomicNumber: integer (nullable = true)
 |-- Element: string (nullable = true)
 |-- Symbol: string (nullable = true)
 |-- AtomicMass: double (nullable = true)
 |-- NumberofNeutrons: integer (nullable = true)
 |-- NumberofProtons: integer (nullable = true)
 |-- NumberofElectrons: integer (nullable = true)
 |-- Period: integer (nullable = true)
 |-- Group: integer (nullable = true)
 |-- Phase: string (nullable = true)
 |-- Radioactive: string (nullable = true)
 |-- Natural: string (nullable = true)
 |-- Metal: string (nullable = true)
 |-- Nonmetal: string (nullable = true)
 |-- Metalloid: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- AtomicRadius: double (nullable = true)
 |-- Electronegativity: double (nullable = true)
 |-- FirstIonization: double (nullable = true)
 |-- Density: double (nullable = true)
 |-- MeltingPoint: double (nullable = true)
 |-- BoilingPoint: double (nullable = true)
 |-- NumberOfIsotopes: integer (nullable = true)
 |-- Discoverer: s

## `pyspark.sql` vs SQL

### Order of execution

![](./notes/img/order.png)

The code below selects the `phrase` column that contain `"liq"`, then runs groupby and count.

SQL equivalent would be:

```sql
SELECT
  period,
  count(*)
FROM elements
WHERE phase = "liq"
GROUP BY period;
```

In [3]:
elements.where(F.col("phase") == "liq").groupby("period").count().show()

+------+-----+
|period|count|
+------+-----+
|     6|    1|
|     4|    1|
+------+-----+



### Using SQL queries on a data frame

- In order to allow a data frame to be queried via SQL, we need to _register_ them as tables.
- Spark SQL does not have visibility over the variables Python assigns.
- Use `createOrReplaceTempView()` to read a data frame and create a Spark SQL reference.  Functionally equivalent to `CREATE_OR_REPLACE_VIEW` in SQL

In [5]:
# Directly querying a data frame SQL-style does not work
try:
    spark.sql(
        "select period, count(*) from elements where phase='liq' group by period"
    ).show(5)
except AnalysisException as e:
    print(e)

Table or view not found: elements; line 1 pos 29;
'Aggregate ['period], ['period, unresolvedalias(count(1), None)]
+- 'Filter ('phase = liq)
   +- 'UnresolvedRelation [elements]



In [12]:
# Using createOrReplaceTempView

elements.createOrReplaceTempView("elements")

spark.sql(
    "select period, count(*) from elements where phase='liq' group by period"
).show(5)

+------+--------+
|period|count(1)|
+------+--------+
|     6|       1|
|     4|       1|
+------+--------+



### Table vs View concept

> In SQL, they are distinct concepts: the table is materialized in memory and the view is computed on the fly. Spark’s temp views are conceptually closer to a view than a table. Spark SQL also has tables but we will not be using them, preferring reading and materializing our data into a data frame.

## Using the Spark catalog for multiple views

- Spark catalog mainly deals with managing metadata of multiple SQL tables, and their level of caching.
- Catalogs manages views we've registered and drops them.

In [13]:
# Instantiate
spark.catalog

# List tables we've registered
display(spark.catalog.listTables())

# Drop a table
spark.catalog.dropTempView("elements")
display(spark.catalog.listTables())

[Table(name='elements', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

[]

### Data Source - Backblaze Data Set

In [ ]:
# Read backblaze data set into a data frame and register a SQL view

DATA_DIRECTORY = "./data/Ch07/"

q1 = spark.read.csv(
    DATA_DIRECTORY + "drive_stats_2019_Q1", header=True, inferSchema=True
)
q2 = spark.read.csv(
    DATA_DIRECTORY + "data_Q2_2019", header=True, inferSchema=True
)
q3 = spark.read.csv(
    DATA_DIRECTORY + "data_Q3_2019", header=True, inferSchema=True
)
q4 = spark.read.csv(
    DATA_DIRECTORY + "data_Q4_2019", header=True, inferSchema=True
)

# Q4 has two more fields than the rest

q4_fields_extra = set(q4.columns) - set(q1.columns)

for i in q4_fields_extra:
    q1 = q1.withColumn(i, F.lit(None).cast(T.StringType()))
    q2 = q2.withColumn(i, F.lit(None).cast(T.StringType()))
    q3 = q3.withColumn(i, F.lit(None).cast(T.StringType()))

# Union the data frames
# if you are using the full set of data, use this version
backblaze_2019 = (
    q1.select(q4.columns)
    .union(q2.select(q4.columns))
    .union(q3.select(q4.columns))
    .union(q4)
)

# Setting the layout for each column according to the schema
q = backblaze_2019.select(
    [
        F.col(x).cast(T.LongType()) if x.startswith("smart") else F.col(x)
        for x in backblaze_2019.columns
    ]
)

# Register the view
backblaze_2019.createOrReplaceTempView("backblaze_stats_2019")